In [1]:
# takes ~20sec
use Dan :ALL;

viz. https://en.wikipedia.org/w/index.php?title=Filmfare_Award_for_Best_Actress&action=edit

In [2]:
my @lines = 'FilmFare.txt'.IO.lines;
.say for @lines[0..20];

===1950s===

{| class="wikitable sortable" rowspan=2 style="text-align: left;" border="2" cellpadding="5"
|-
!scope="col" style="width:3%; text-align:center;"| Year
!scope="col" style="width:3%;text-align:center;"| Photos of winners
!scope="col" style="width:15%;text-align:center;"| Actress
!scope="col" style="width:15%;text-align:center;"| Role(s)
!scope="col" style="width:15%;text-align:center;"| Film
|-
! scope="row" rowspan=2 style="text-align:center" | 1954 <br /><small>[[1st Filmfare Awards|(1st)]] </small>
|rowspan=2 style="text-align:center"|{{border|[[File:Meena Kumari.jpg|95px|centre]]|width=2px|style=solid |color=gold}}
| style="background:#FAEB86;" |'''{{sort|Meena|[[Meena Kumari]]}}''' {{double dagger|alt=Award winner}}
| style="background:#FAEB86;" | '''{{sort|Gauri|Gauri}}'''
| style="background:#FAEB86;" | '''''[[Baiju Bawra (film)|Baiju Bawra]]'''''
|-
| colspan =3 style="text-align:center" | '''No Other Nominee'''
|-
! scope="row" rowspan=2 style="text-align:center" |

### PART I: USE REGEXES TO EXTRACT DATA ITEMS

In [3]:
my @exclusions = <Year|Photos|Actress|Role|Film|Filmfare|Awards|File|No|Other|Nominee|Portrait|Returns>;

my regex tcword {       # tite case word
    <upper>             # first character uppercase 
    <lower>+            # rest of chars lowercase
}

my regex name {
    <tcword>+ %% ' '    # one or more separated by a space
}

my regex year {         
    \d**4               # four consecutive digits
}

my ( $current, @years, @names );

for @lines -> $l is rw {
    next if $l ~~ /File/;               # avoid getting name from photo links

    $l ~~ s:g/<@exclusions>//;          # strip out exclusions

    if $l ~~ /<year>/ {
        $current = $<year>;             # current year is 'sticky'
    }

    if $l ~~ /<name>/ {
        @years.push: $current;
        @names.push: ~$<name>.trim;
    }
}
@names[0..10].join("::").say;

Meena::Gauri::Baiju Bawra::Meena::Lalita::Parineeta::Kamini::Biraj::Biraj Bahu::Geeta::Kamala


### PART II: MAP DATA ITEMS TO 2-D ARRAY AND MAKE DATAFRAME 

In [4]:
my @name3 = @names.rotor(3);            # rotor name over cols 1..3
my @year3 = @years.rotor(3);            # match year cadence to names

my @data;                               # 2d array

for 0..@year3 -> $m {
    next unless @year3[$m][0];
    
    @data[$m;0] = @year3[$m][0];
    @data[$m;1..3] = @name3[$m];
}

my @columns = <Year Actor Role Film>;

my \df = DataFrame.new( :@data, :@columns );
say ~df[0..5]^;


    Year  Actor   Role    Film        
 0  1954  Meena   Gauri   Baiju Bawra 
 1  1955  Meena   Lalita  Parineeta   
 2  1956  Kamini  Biraj   Biraj Bahu  
 3  1956  Geeta   Kamala  Vachan      
 4  1956  Meena   Shobha  Azaad       
 5  1957  Nutan   Gauri   Seema       


### PART III: HISTOGRAM FOR OUTLIERS

In [38]:
my \abh = df[*]<Actor>.data.BagHash;
abh.^name;

BagHash

In [37]:
for abh.sort(-*.value).[0..^40] { say .key.fmt('%-20s'), '+' xx .value }

Meena               (+ + + + + + + + + +)
Rani                (+ + + + + + + +)
Aishwarya           (+ + + + + + + +)
Aarti               (+ + + + +)
Preity              (+ + + + +)
Vidya               (+ + + +)
Karisma             (+ + + +)
Kareena             (+ + + +)
Kajol               (+ + + +)
Tabu                (+ + +)
Madhu               (+ + +)
Nutan               (+ + +)
Vyjayanthimala      (+ + +)
Urmila              (+ + +)
Hema                (+ + +)
Meera               (+ +)
Kamini              (+ +)
Radha               (+ +)
Champa              (+ +)
Judaai              (+ +)
Nisha               (+ +)
Ameesha             (+ +)
Shanti              (+ +)
Shivani             (+ +)
Geeta               (+ +)
Saira               (+ +)
Bipasha             (+ +)
Zoya                (+ +)
Pooja               (+ +)
Lakshmi             (+ +)
Madhuri             (+ +)
Deepika             (+ +)
Madhavi             (+ +)
Jeevan Dhaara       (+)
Benazir             (+)
Prem Rog      

### PART IV: TEXT PLOT FOR OUTLIERS

zef install [Text::Plot](https://raku.land/zef:antononcube/Text::Plot) from Anton (nice!)

In [5]:
my \ybh = df[*]<Year>.data.BagHash;
ybh.^name;

BagHash

In [6]:
use Text::Plot;

In [7]:
say text-list-plot([ybh.keys.map(+*)], [ybh.values],
                    point-char => '+',
                    x-tick-labels-format => '%d', 
                    y-tick-labels-format => '%d',
                   );

+-+------------+------------+-------------+------------+---+ 
|                                                          | 
+                                                  + + +   +6
|                                                          | 
+                               +  ++ + + +      +    +    +5
|                                                          | 
+                                     +      ++ +  +   +   +4
|                                                          | 
+            +   + ++   + +++++ + ++    + +++  +++ ++      +3
|                                                          | 
+              +  + ++ +++  + ++    +++++ ++++ + ++        +2
|                                                          | 
+   +      +++++++++ ++++++++++ +   + + ++  ++++ + +       +1
|                                                          | 
+-+------------+------------+-------------+------------+---+ 
  1940         1960         1980          2000         2020  
